In [ ]:
from cobra.io.yaml import from_yaml
from cobra.io.sbml import write_sbml_model
import geckopy

In [ ]:
yeast_ecgem_yaml = r"/home/daiki/Github/ENEOS/python/FBA/models/ecYeastGEM (SysBioChalmersGECKO).yml"
yeast_ecgem_xml = r"/home/daiki/Github/ENEOS/python/FBA/models/ecYeastGEM (SysBioChalmersGECKO).xml"

# YAMLを読み込み
yaml_model = from_yaml(open(yeast_ecgem).read())
# SBMLで保存
write_sbml_model(yaml_model, yeast_ecgem_xml)
# geckopyでSBMLを読み込み
ec_model = geckopy.io.read_sbml_ec_model(yeast_ecgem_xml)